In [13]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('FER/fer2013.csv')

# Filter the dataset into three sets based on the value in 'Usage'
training_set = df[df['Usage'] == 'Training']
testing_set = df[df['Usage'] == 'PublicTest']
validation_set = df[df['Usage'] == 'PrivateTest']

emotion_mapping = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

# Display the first few rows of each set
print("Training Set:")
print(training_set.head())

print("\nTesting Set:")
print(testing_set.head())

print("\nValidation Set:")
print(validation_set.head())

Training Set:
   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training

Testing Set:
       emotion                                             pixels       Usage
28709        0  254 254 254 254 254 249 255 160 2 58 53 70 77 ...  PublicTest
28710        1  156 184 198 202 204 207 210 212 213 214 215 21...  PublicTest
28711        4  69 118 61 60 96 121 103 87 103 88 70 90 115 12...  PublicTest
28712        6  205 203 236 157 83 158 120 116 94 86 155 180 2...  PublicTest
28713        3  87 79 74 66 74 96 77 80 80 84 83 89 102 91 84 ...  PublicTest

Validation Set:
       emotion                                         

In [29]:
import os
from PIL import Image
import matplotlib.pyplot as plt

destination_directory = 'Fer_ResEmoteNet'

# Create directories if they don't exist
os.makedirs(destination_directory+'/train', exist_ok=True)
os.makedirs(destination_directory+'/test', exist_ok=True)
os.makedirs(destination_directory+'/validation', exist_ok=True)

def save_images(dataset, set_name):
    for index, row in dataset.iterrows():
        emotion_label = emotion_mapping[row['emotion']]
        image_data = row['pixels']
        image_pixels = np.fromstring(image_data, sep=' ').astype(int)
        image_pixels = image_pixels.reshape(48, 48)
        
        # Create an image from the pixel data
        image = Image.fromarray(image_pixels.astype('uint8'), 'L')
        # plt.imshow(image, cmap='gray')
        # plt.show()
        
        # Save the image to the appropriate directory
        image.save(f'{destination_directory}/{set_name}/{set_name}_{index}_{emotion_label.lower()}.jpg')

# Save images for each set
save_images(training_set, 'train')
save_images(testing_set, 'test')
save_images(validation_set, 'validation')

In [34]:
import shutil
file_extension = 'jpg'

label_mapping = {'angry':0, 'disgust':1, 'fear':2, 'happy':3, 'sad':4, 'surprise':5, 'neutral':6}

folder_mapping = {'test': 'test', 'train': 'train', 'validation': 'val'}

# Iterate over the folders (test, train, validation)
for folder in ['test', 'train', 'validation']:
    folder_path = os.path.join(destination_directory, folder)

    image_data = []

    # Make sure the name of the file is partition_iteration_emotion.jpg or .png
    for filename in os.listdir(folder_path):
        if filename.endswith(file_extension):  
            label_name = filename.split('_')[-1].split('.')[0]
            label_value = label_mapping.get(label_name)
            if label_value is not None:  
                image_data.append([filename, label_value])

    df = pd.DataFrame(image_data, columns=["ImageName", "Label"])

    csv_file_path = os.path.join(folder_path, f'{folder}_labels.csv')

    df.to_csv(csv_file_path, index=False, header=False)